<a href="https://colab.research.google.com/github/pvoipt/AiDeveloping/blob/main/TickNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical  # one-hot encoding 을 위한 함수
from tensorflow.keras.models import Sequential  # 레이어를 층층히 쌓아가는 연쇄 모델
from tensorflow.keras.layers import Dense  # 완전연결층
from tensorflow.keras.models import load_model  # 저장된 모델 불러오기

"""퍼셉트론 설계
      데이터 처리 용이함을 위해 행렬 사용

   신경망 설계
      각 퍼셉트론에서 다음 퍼셉트론으로 값을 전달하는 것을 시간 1로 침(모든 퍼셉트론이 동시에 작동하게 하기 위함)
      작동 방법
        1. 지정 숫자만큼 퍼셉트론 생성
        2. 퍼셉트론별로 

   퍼셉트론 클래스, 신경망 생성 클래스를 따로
"""

In [2]:
def stepF(X):
  if np.sum(X) > 0.3:
    return 1
  else:
    return 0

In [3]:
def ReLU(X):
  return np.maximum(0, np.sum(X))

In [4]:
def SSE(y, t):    #sum of squares for error, 오차제곱합
  return 0.5 * np.sum((y - t)**2)

In [5]:
class Perceptron:
  def __init__(self, size, nSpN, role, sumType, nextSize):
    self.role = role
    if role == "sensory":       #감각뉴런이라면 가중치 1 (원본 그대로 반영)
      self.wgt = np.ones(100)
    elif role == "inter":       #연결뉴런이라면 가중치 0~1 랜덤
      self.wgt = np.random.rand(100)
    elif role == "motor":
      pass
    self.size = size
    if nextSize == -1:
      self.nextSize = size
    else:
      self.nextSize = nextSize
    self.sumType = sumType
    self.nSpN = nSpN    # nSpN = the number of Synapse per Neuron : 뉴런별로 뻗어나갈 시냅스 수
    self.X = []       #입력값 받는 리스트
    self.summation = 0      #가중치을 더하는 변수 (시간적 가중 용)
  def connectNeuron(self):
    self.cN = np.random.randint(0, self.nextSize, self.nSpN)    #랜덤하게 시냅스 연결
  def input_(self, X):    #값 입력받음
    self.X += X
  def out(self):
    self.WX = self.X*self.wgt[:len(self.X)]   #가중치와 입력 곱함
    if self.role == "inter":
      self.X = []
    if self.role == "motor":
      return 0
    else:
      if self.sumType == "spatial":      #공간적/시간적 가중에 따라 활성화 방식 다름
        return stepF(self.WX)
      elif self.sumType == "temporal":
        self.summation += np.sum(self.WX)
        if self.summation >= 1:
          self.summation -= 1
          return 1
        else:
          return 0

In [6]:
class Network:
  def __init__(self, size, nSpN, nInputLayer, nOutputLayer): #nSpN = the number of Synapse per Neuron
    self.size = size
    self.nSpN = nSpN
    self.inputLayer = []
    self.neuron = []
    self.nInputLayer = nInputLayer
    self.nOutputLayer = nOutputLayer
    self.inputTable = []
  def newNeuron(self, list_, role, nextSize):
    n = Perceptron(self.size, self.nSpN, role, "temporal", nextSize)
    n.connectNeuron()
    list_.append(n)
  def makeup(self):
    for i in range(self.nInputLayer):         #입력뉴런(감각뉴런) 생성
      n = Perceptron(self.size, self.nSpN, "sensory", "temporal", self.size)
      n.connectNeuron()
      self.inputLayer.append(n)
    for i in range(self.size):      #연결뉴런 생성
      n = Perceptron(self.size, self.nSpN, "inter", "temporal", self.size + self.nOutputLayer)
      n.connectNeuron()
      self.neuron.append(n)
    for i in range(self.nOutputLayer):
      n = Perceptron(self.size, self.nSpN, "motor", "temporal", -1)
      self.neuron.append(n)
  def input_(self):     #입력층 입력
    for i in range(self.size + self.nOutputLayer):
      self.inputTable.append([])
    for i in range(self.nInputLayer):
      r = self.inputLayer[i].out()
      for j in range(self.nSpN):
        self.inputTable[self.inputLayer[i].cN[j]].append(r)
  def tick(self):       #시간 경과 (틱)
    for i in range(self.size):
      r = self.neuron[i].out()
      for j in range(self.nSpN):
        self.inputTable[self.neuron[i].cN[j]].append(r)
        
    for i in range(self.size + self.nOutputLayer):
      if len(self.inputTable[i]):
        self.neuron[i].input_(self.inputTable[i])
    self.inputTable = []
    for i in range(self.size + self.nOutputLayer):
      self.inputTable.append([])
  def output(self):     #출력층 출력
      self.out = []
      for i in range(self.size, self.size + self.nOutputLayer):
        self.out.append(np.sum(self.neuron[i].X))
      return self.out / np.sum(self.out)

In [7]:
#MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

input_shape = X_train.shape[1] * X_train.shape[2] #그림 크기 (28*28)
number_of_classes = len(set(y_train))

X_train = X_train / 255.0  #정규화
X_test = X_test / 255.0
X_train = X_train.reshape(-1, input_shape) #차원 낮추기
X_test = X_test.reshape(-1, input_shape)

y_train = to_categorical(y_train, number_of_classes) #원-핫 인코딩
y_test = to_categorical(y_test, number_of_classes)

In [8]:
print(np.shape(X_train))

X_train[0][550]

(60000, 784)


0.9803921568627451

In [9]:
n = []
total = []
for a in range(10):
  n.append(Network(10000, 10, 784, 10))
  n[a].makeup()
  t = 0
  for i in range(10):
    for j in range(784):
      n[a].inputLayer[j].input_([X_train[i][j]])
    for k in range(5):
      n[a].input_()
      n[a].tick()
    t += SSE(y_train[i], n[a].output())
  total.append(t)
  
    


In [11]:
total

[4.508515946680563,
 4.6129597813861904,
 4.657810866312001,
 4.663671013819764,
 4.569778191072963,
 4.444110020683776,
 4.929257394546239,
 4.855791686942335,
 4.494827995198796,
 4.581200732956355]